# Intake-Postgres Plugin: PostGIS Demo

The following notebook demonstrates PostGIS functionality using the _Intake-Postgres_ plugin. Its purpose is to showcase a variety of scenarios in which an _Intake_ user may want to query their PostgreSQL-based relational datasets.

PostGIS functionality is to be exercised with the following geometry objects: `POINT, LINESTRING, POLYGON, MULTIPOINT, MULTILINESTRING, MULTIPOLYGON`.


## Setup
1. With [Docker installed](https://www.docker.com/community-edition), execute:
    ```
    docker run -d -p 5432:5432 --name intake-postgres-db mdillon/postgis:9.6-alpine
    ```
    All subsequent `docker run` commands will start containers from this image.

1. In the same conda environment as this notebook, install `pandas`, `sqlalchemy`, `psycopg2`, `shapely`, and (optionally) `postgresql`:
    ```
    conda install pandas sqlalchemy psycopg2 shapely postgresql
    ```
    The `postgresql` package is only for the command-line client library, so that we can verify that results were written to the database (externally from our programs).

1. Finally, install the _intake-postgres_ plugin:
    ```
    conda install -c intake intake-postgres
    ```


## Loading the data

Because _Intake_ only supports reading the data, we need to insert the data into our databases by another means. The general approach below relies on inserting the data from a GeoJSON into separate tables.

The code (below) begins by importing the necessary modules:

In [ ]:
from __future__ import print_function, absolute_import

## For loading / creating test data
import json
import os
import requests
from shapely.geometry import (Point, MultiPoint,
                              LineString, MultiLineString,
                              Polygon, MultiPolygon)

## For inserting test data
import pandas as pd
from sqlalchemy import create_engine

## For using Intake
from intake.catalog import Catalog

## For data viz
from shapely import wkt
import numpy as np
import bokeh.plotting as bp
from bokeh.models import ColumnDataSource
from bokeh.models import WMTSTileSource, HoverTool
from bokeh.io import output_notebook
output_notebook()

%matplotlib inline

Here we download the data, if it doesn't already exist:

In [ ]:
%%time

# Define download sources and destinations.
url = 'https://hifld-dhs-gii.opendata.arcgis.com/datasets/e277657582f74ed78dc2a503eae7fa2e_0.geojson'
fpath = 'fortune_500.geojson'

# More about this dataset:
# https://hifld-dhs-gii.opendata.arcgis.com/datasets/e277657582f74ed78dc2a503eae7fa2e_0

# Do the data downloading
if os.path.isfile(fpath):
    print('{!r} already exists: skipping download.\n'.format(fpath))
else:
    try:
        print('Downloading data from {!r}...'.format(url))
        response = requests.get(url)
    except:
        raise ValueError('Download error. Check internet connection and URL.')

    try:
        with open(fpath, 'wb') as fp:
            print('Writing data...'.format(fpath))
            fp.write(response.content)
    except:
        raise ValueError('File write error. Check destination file path and permissions')

    print('Success: {!r}\n'.format(fpath))

Next, we load the data and save it to the database as PostGIS objects:

In [ ]:
%time

import psycopg2
conn = psycopg2.connect(host='localhost', dbname='postgres', user='postgres', port=5432)
# conn.set_isolation_level(1)

with open(fpath, 'r') as fp:
    data = json.load(fp)

cursor = conn.cursor()
cursor.execute('drop table if exists fortune500')
cursor.execute('create table fortune500 (location geometry(point, 4326), name text, address text, city text, state text, zip_code text, county text, directions text)')
for feature in data['features']:
    lon, lat = feature['geometry']['coordinates']
    props = feature['properties']
    zip_code, state, name, address, county, city, directions = props['ZIP'], props['STATE'], props['NAME'], props['ADDRESS'], props['COUNTY'], props['CITY'], props['DIRECTIONS']
    cursor.execute('insert into fortune500 values (St_GeomFromText(%s), %s, %s, %s, %s, %s, %s, %s)',
        ('SRID=4326;POINT({} {})'.format(lon, lat), name, address, city, state, zip_code, county, directions),
    )
conn.commit()
conn.close()

Verify the data was written, by connecting to the databases directly with the `psql` command-line tool:

In [ ]:
# Save each query from the `psql` command as HTML
!psql -h localhost -p 5432 -U postgres -q -H -c "select * from fortune500 limit 5;" > points.html

# Display the HTML files
from IPython.display import display, HTML
display(HTML('points.html'))

## Reading the data (with Intake-Postgres)

Write out a __gis\_catalog.yml__ file with the appropriate schema:

In [ ]:
%%writefile gis_catalog.yml
plugins:
  source:
    - module: intake_postgres

sources:
  fortune500:
    driver: postgres
    args:
      uri: 'postgresql://postgres@localhost:5432/postgres'
      sql_expr: |
          select * from fortune500

Access the catalog with Intake:

In [ ]:
%time
catalog = Catalog('gis_catalog.yml')
catalog

## Geometry (point) data

Here we inspect the data source metadata, read the data, and visualize it.

In [ ]:
catalog.fortune500.discover()

Read the data from the source:

In [ ]:
%time
points = catalog.fortune500.read()
points.tail()

Next, we prepare the data for visualization. With the help of `numpy` and `shapely`, we can convert latitude / longitude coordinates into mercator (UTM), and pass the mercator coordinates to Bokeh for visualization:

In [ ]:
%time

# Earth's radius, in meters
EARTH_RADIUS = 6378137.

def lon_to_mercx(lon):
    x = EARTH_RADIUS * np.radians(lon)
    return x

def lat_to_mercy(lon, lat):
    x = lon_to_mercx(lon)
    y = 180. / np.pi * np.log(np.tan(np.pi / 4. + lat * (np.pi / 180.) / 2.)) * x / lon
    return y

def wkt_to_x(pt):
    return lon_to_mercx(wkt.loads(pt).x)

def wkt_to_y(pt):
    point = wkt.loads(pt)
    return lat_to_mercy(point.x, point.y)

points['y'] = np.vectorize(wkt_to_y)(points['location'].values)
points['x'] = np.vectorize(wkt_to_x)(points['location'].values)
points.tail()

Next, we visualize the data interactively with `bokeh`:

In [ ]:
points_source = ColumnDataSource(points)
tilesource = WMTSTileSource(url='http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png')
p = bp.figure(width=800)
p.axis.visible = False
p.add_tile(tilesource)
hover_tool = HoverTool(tooltips=[('Name', '@name'), ('Address', '@address, @city, @state, @zip_code')])
p.add_tools(hover_tool)
p.circle('x', 'y', source=points_source, size=10, fill_alpha=.2)
bp.show(p)